In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pinecone
from pinecone import Pinecone
from pinecone import Pinecone, ServerlessSpec

In [2]:
# Cargar los datos de juegos (descripciones)
df_juegos = pd.read_csv('../data_extraction/boardgames_10000_juegos.csv')

In [3]:
df_juegos.columns

Index(['BGGId', 'Name', 'Year_Published', 'Description', 'Min_Players',
       'Max_Players', 'Min_Playtime', 'Max_Playtime', 'Average_Rating',
       'Bayesian_Average_Rating', 'Number_of_Ratings', 'Mechanics',
       'Categories'],
      dtype='object')

In [4]:
# Obtener la descripción de los juegos
df_descripciones = df_juegos[['BGGId','Name','Description', 'Mechanics', 'Categories', 'Average_Rating', 'Bayesian_Average_Rating']]  

In [5]:
df_descripciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BGGId                    10000 non-null  int64  
 1   Name                     10000 non-null  object 
 2   Description              9992 non-null   object 
 3   Mechanics                10000 non-null  object 
 4   Categories               10000 non-null  object 
 5   Average_Rating           10000 non-null  float64
 6   Bayesian_Average_Rating  10000 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 547.0+ KB


In [6]:
df_descripciones.dropna(inplace=True)
df_descripciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9992 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BGGId                    9992 non-null   int64  
 1   Name                     9992 non-null   object 
 2   Description              9992 non-null   object 
 3   Mechanics                9992 non-null   object 
 4   Categories               9992 non-null   object 
 5   Average_Rating           9992 non-null   float64
 6   Bayesian_Average_Rating  9992 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 624.5+ KB


C:\Users\Laura Ortiz\AppData\Local\Temp\ipykernel_12820\1111501472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_descripciones.dropna(inplace=True)


In [7]:
df_descripciones.sample(5)

,BGGId,Name,Description,Mechanics,Categories,Average_Rating,Bayesian_Average_Rating
9684,38482,Freaky Forest: The Halloween Game,From the Black & White Games website:&#10;&#10...,['Dice Rolling'],['Dice'],5.87500,0.00000
5423,252055,Drawn from the Dark,From publisher blurb:&#10;&#10;Drawn from the ...,[],[],7.00000,0.00000
4193,217018,Frankie's Food Truck Fiasco Game,Description from the publisher:&#10;&#10;Order...,['Set Collection'],"[""Children's Game"", 'Educational']",5.90500,0.00000
1361,44629,Chaining the Beast,WW2432: Chaining the Beast&#10;&#10;A sourcebo...,[],[],5.85714,5.58141
1466,74217,Battletoads in Battlemaniacs,"""Battletoads in Battlemaniacs is a game releas...",[],[],6.07143,5.55899


In [8]:
# Asegúrate de descargar stopwords y wordnet de nltk
nltk.download('stopwords')
nltk.download('wordnet')

# Función mejorada de preprocesamiento de texto
def preprocesar_descripcion(texto):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    # Eliminar entidades HTML y otros caracteres no deseados
    texto = re.sub(r'&#?\w+;', ' ', texto)  # Reemplazar entidades HTML como &#10; 
    texto = re.sub(r'\W+', ' ', texto)  # Eliminar caracteres no alfanuméricos
    texto = re.sub(r'\d+', '', texto)  # Eliminar números
    
    # Convertir a minúsculas, eliminar stopwords y lematizar (solo si el texto es válido)
    if isinstance(texto, str):
        palabras = [lemmatizer.lemmatize(palabra) for palabra in texto.lower().split() if palabra not in stop_words]
        return " ".join(palabras)
    else:
        return ""  # Devolver cadena vacía si no es un texto válido



[nltk_data] Downloading package stopwords to C:\Users\Laura
[nltk_data]     Ortiz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Laura
[nltk_data]     Ortiz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
# Aplicar el preprocesamiento mejorado a las descripciones
df_descripciones['Descripcion_limpia'] = df_descripciones['Description'].apply(preprocesar_descripcion)

# Convertir listas de mecánicas y categorías en cadenas de texto
df_descripciones['Mecanicas'] = df_descripciones['Mechanics'].apply(lambda x: ', '.join(x))
df_descripciones['Categorias'] = df_descripciones['Categories'].apply(lambda x: ', '.join(x))

# Combinar la descripción limpia con las mecánicas y categorías
df_descripciones['Descripcion_completa'] = df_descripciones['Descripcion_limpia'] + " " + df_descripciones['Mecanicas'] + " " + df_descripciones['Categorias']

C:\Users\Laura Ortiz\AppData\Local\Temp\ipykernel_12820\1354438255.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_descripciones['Descripcion_limpia'] = df_descripciones['Description'].apply(preprocesar_descripcion)
C:\Users\Laura Ortiz\AppData\Local\Temp\ipykernel_12820\1354438255.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_descripciones['Mecanicas'] = df_descripciones['Mechanics'].apply(lambda x: ', '.join(x))
C:\Users\Laura Ortiz\AppData\Local\Temp\ipykernel_12820\1354438255.py:6: Setti

In [10]:
# Entrenar Doc2Vec usando los índices del DataFrame como etiquetas
documents = [TaggedDocument(doc.split(), [str(idx)]) for idx, doc in zip(df_descripciones.index, df_descripciones['Descripcion_completa'])]
model = Doc2Vec(documents, vector_size=50, window=2, min_count=1, workers=4)

In [11]:
# Inicializar Pinecone
pc = Pinecone(api_key="e2659d52-b976-4624-b8b8-8de36f8ea15a")

# Nombre del índice que vamos a crear
index_name = "boardgames-recommendation"

# Crear el índice con la nueva API
pc.create_index(
    name=index_name,
    dimension=50,  # La dimensión de tus embeddings (en este caso, de Doc2Vec)
    metric="cosine",  # Similaridad del coseno para las búsquedas
    spec=ServerlessSpec(
        cloud="aws",       # Proveedor de nube
        region="us-east-1"  # Región, asegúrate de poner la que corresponda a tu entorno
    )
)



In [12]:
# Conectar al índice existente
index = pc.Index(index_name)

# Subir los embeddings de los juegos a Pinecone
for idx, row in df_descripciones.iterrows():
    embedding_vector = model.dv[str(idx)]  # El embedding generado por Doc2Vec
    game_id = row['BGGId']  # Usamos el ID del juego como identificador
    
    # Insertar el embedding en Pinecone
    index.upsert([(str(game_id), embedding_vector.tolist())])

print("Embeddings subidos a Pinecone correctamente.")

Embeddings subidos a Pinecone correctamente.


In [13]:
# Función para recomendar juegos utilizando Pinecone y valoraciones ponderadas
def recomendar_juegos_pinecone_con_valoraciones(prompt, model, index, df_juegos, top_n=5, w=0.7):
    # Generar el embedding del prompt usando Doc2Vec
    prompt_vector = model.infer_vector(prompt.split())

    # Realizar la consulta en Pinecone
    result = index.query(vector=prompt_vector.tolist(), top_k=top_n, include_values=False)

    # Obtener las recomendaciones
    recomendaciones = []
    for match in result['matches']:
        game_id = match['id']
        score = match['score']

        # Buscar el juego en el DataFrame por su ID
        game_row = df_juegos[df_juegos['BGGId'] == int(game_id)]
        if not game_row.empty:
            valoracion = game_row['Average_Rating'].values[0] / 10  # Normalizamos la valoración (si está en rango 0-10)
            similaridad_ponderada = (score * w) + (valoracion * (1 - w))
            recomendaciones.append((game_row['Name'].values[0], similaridad_ponderada, score, valoracion))

    # Ordenar por similaridad ponderada
    recomendaciones.sort(key=lambda x: x[1], reverse=True)

    # Mostrar los resultados
    print("Juegos recomendados:")
    for rec in recomendaciones:
        print(f"{rec[0]} (Similaridad ponderada: {rec[1]:.4f}, Similaridad: {rec[2]:.4f}, Valoración: {rec[3]:.2f})")



In [14]:
# Prompt del usuario
prompt_usuario = 'I love solo games with rich storytelling and immersive themes.'

# Ejecutar la recomendación
recomendar_juegos_pinecone_con_valoraciones(prompt_usuario, model, index, df_juegos)

Juegos recomendados:
A Bagel Break (Similaridad ponderada: 0.4072, Similaridad: 0.5817, Valoración: 0.00)
Party Time with Winnie the Pooh (Similaridad ponderada: 0.3932, Similaridad: 0.5618, Valoración: 0.00)
Alpha Blast (Similaridad ponderada: 0.3896, Similaridad: 0.5566, Valoración: 0.00)
Limbo: The Passing (Similaridad ponderada: 0.3816, Similaridad: 0.5451, Valoración: 0.00)
Köpcentrum (Similaridad ponderada: 0.3767, Similaridad: 0.5381, Valoración: 0.00)
